In [1]:
import pandas as pd
import numpy as np


In [2]:
myFile = pd.read_csv('D:/project/550_Pattern/Project_1/550-p1-cset-krk-1.csv')
myFile
myFrame=pd.DataFrame(myFile.values, columns = ["White-King-File", "White-King-rank", "White-Rook-file", "White-Rook-rank","Black-King-file","Black-King-rank","Class"])
myFrame

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
0,c,1,e,2,d,3,draw
1,b,1,d,7,c,7,draw
2,d,2,g,1,g,2,draw
3,d,4,g,6,f,6,draw
4,c,1,b,5,a,5,draw
...,...,...,...,...,...,...,...
214,d,3,d,7,a,7,nine
215,b,1,b,4,g,1,nine
216,d,4,e,8,g,2,nine
217,c,2,d,5,f,1,nine


In [3]:
myFrame.shape

(219, 7)

In [4]:
# Split into training set:60%, holdout set:20%, Validation set: 20%

training_set = myFrame.sample(frac=0.6)
remaining_set = myFrame.drop(training_set.index)
training_set

,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
115,d,3,c,7,b,1,five
160,d,2,f,6,h,1,seven
124,d,1,e,4,b,1,five
102,d,3,g,2,b,1,four
180,c,2,c,5,b,8,eight
...,...,...,...,...,...,...,...
118,d,4,d,2,f,1,five
30,c,2,d,7,a,1,one
82,c,1,b,4,a,1,three
157,c,2,f,3,h,2,seven


In [5]:
remaining_set


,White-King-File,White-King-rank,White-Rook-file,White-Rook-rank,Black-King-file,Black-King-rank,Class
1,b,1,d,7,c,7,draw
4,c,1,b,5,a,5,draw
7,d,2,g,2,f,3,draw
9,b,1,g,5,f,6,draw
13,a,1,h,4,g,4,draw
...,...,...,...,...,...,...,...
209,d,4,b,6,e,1,nine
213,d,4,d,8,g,1,nine
214,d,3,d,7,a,7,nine
216,d,4,e,8,g,2,nine


In [6]:
classes, cls_counts = np.unique(training_set['Class'], return_counts = True)
classes, cls_counts


(array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
        'three', 'two', 'zero'], dtype=object),
 array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))

In [7]:
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [8]:
calculate_entropy(training_set['Class'])

3.408507643499304

In [9]:

for cols in training_set.columns[:-1]:
    print(cols)

White-King-File
White-King-rank
White-Rook-file
White-Rook-rank
Black-King-file
Black-King-rank


In [10]:
# dataset = training_set
# feature = Feature or attribute of the dataset, i.e column name (training_set.columns[:-1]) E.g White-King-File
# label = Name of Class label colums, i.e. 'Class'

def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy    
    return feature_info_gain

In [11]:
calculate_information_gain(training_set, 'White-King-File', 'Class')

0.4832861416161367

In [16]:
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label], return_counts=True)
    unique_data = np.unique(dataset[label])
#     print('datum',datum)
#     print('unique_data',unique_data)
    if len(unique_data)<=1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]
    elif len(features) == 0:
        return parent
    else:
        parent = unique_data[np.argmax(datum[1])]
#         print('datum[1]',datum[1])
#         print("parent",parent)
        
        item_values =  [calculate_information_gain(dataset, feature, label) for feature in features]
        optimal_feature_index = np.argmax(item_values)
        optimal_feature = features[optimal_feature_index]
        decision_tree = {optimal_feature:{}}
        features = [i for i in features if i!=optimal_feature]
        
#         print(item_values)
        for value in np.unique(dataset[optimal_feature]):
            min_data = dataset.where(dataset[optimal_feature] == value).dropna()
            
            min_tree = create_decision_tree(min_data, df, features, label, parent)
            
            decision_tree[optimal_feature][value] = min_tree
        
        return (decision_tree)
        

In [17]:
features = training_set.columns[:-1]
label = 'Class'
parent=None
features

Index(['White-King-File', 'White-King-rank', 'White-Rook-file',
       'White-Rook-rank', 'Black-King-file', 'Black-King-rank'],
      dtype='object')

In [18]:
decision_tree = create_decision_tree(training_set,training_set,features,label,parent)
# decision_tree


datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw' 'eight' 'five' 'four' 'nine' 'one' 'seven' 'six' 'three' 'two'
 'zero']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw' 'eight' 'five' 'four' 'one' 'seven' 'six' 'three' 'two' 'zero']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['zero']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['eight' 'one' 'seven' 'six' 'three']
datum (array(['draw', 'eight', 'five', 'f

datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['five']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['four' 'three']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['three']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['four']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['fou

datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw' 'four']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['four']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['four'

datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['seven']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw' 'six']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['six']
datum (array(['draw', 'eight', 'five', 'four', 'nine', 'one', 'seven', 'six',
       'three', 'two', 'zero'], dtype=object), array([13, 15, 11, 15, 13, 15,  8, 11, 12, 13,  5]))
unique_data ['draw']